In [8]:
import os
os.environ["OPENAI_API_KEY"] = key  # reemplaza con tu clave real si es necesario





# Capítulo 6. Arquitectura de Agentes

Basándonos en las arquitecturas descritas en el Capítulo 5, este capítulo cubrirá lo que quizá sea la más importante de todas las arquitecturas actuales de modelos de lenguaje grande (LLM): la arquitectura de agentes. Primero, presentaremos qué hace únicos a los agentes con LLM, luego mostraremos cómo construirlos y cómo extenderlos para casos de uso comunes.

En el campo de la inteligencia artificial, existe una larga historia en la creación de agentes (inteligentes), que pueden definirse de forma sencilla como “algo que actúa”, en palabras de Stuart Russell y Peter Norvig en su libro *Artificial Intelligence* (Pearson, 2020). La palabra *actúa* tiene más implicaciones de lo que parece a simple vista:

- Actuar requiere cierta capacidad para decidir qué hacer.
- Decidir qué hacer implica tener más de una posible acción. Después de todo, una decisión sin opciones no es una decisión.
- Para decidir, el agente también necesita información sobre el entorno externo (cualquier cosa fuera del propio agente).

Así que una aplicación LLM con arquitectura de agente debe ser aquella que utiliza un modelo LLM para elegir entre una o más posibles acciones, dadas ciertas condiciones del estado actual del mundo o un estado deseado. Estas capacidades suelen implementarse combinando dos técnicas de "prompting" que ya conocimos en el prefacio:

**Uso de herramientas**  
Incluye en el prompt una lista de funciones externas que el LLM puede utilizar (es decir, las acciones que puede decidir tomar), junto con instrucciones sobre cómo debe formatear su elección en la salida. En breve veremos cómo se ve esto en un prompt.

**Cadena de razonamiento (chain-of-thought)**  
Los investigadores han descubierto que los LLMs “toman mejores decisiones” cuando se les da instrucciones para razonar sobre problemas complejos dividiéndolos en pasos secuenciales. Esto suele hacerse con frases como “piensa paso a paso” o mostrando ejemplos de preguntas divididas en varios pasos o acciones.

Aquí tienes un ejemplo de prompt que combina el uso de herramientas y la cadena de razonamiento:

---

**Herramientas:**  
- `search`: esta herramienta acepta una consulta de búsqueda web y devuelve los resultados principales.  
- `calculator`: esta herramienta acepta expresiones matemáticas y devuelve su resultado.  

Si deseas usar herramientas para llegar a la respuesta, escribe la lista de herramientas e insumos en formato CSV, con la fila de encabezado: `tool,input`.

Piensa paso a paso; si necesitas usar varias herramientas para llegar a la respuesta, devuelve solo la primera.

**Pregunta:**  
¿Cuántos años tenía el 30º presidente de los Estados Unidos cuando murió?

**Respuesta esperada:**  
tool,input

---

Y la salida, cuando se ejecuta con `gpt-3.5-turbo` a temperatura 0 (para asegurar que el modelo siga el formato CSV esperado) y con salto de línea como secuencia de parada (lo que indica al modelo que deje de generar texto al llegar a ese carácter). Esto hace que el LLM produzca una sola acción (como se esperaba, dado el prompt):

```
search,30th president of the United States
```

Los modelos más recientes han sido ajustados para mejorar su desempeño en tareas de uso de herramientas y cadena de razonamiento, eliminando la necesidad de instrucciones específicas en el prompt.

---

## El Bucle Planificar-Hacer (Plan-Do Loop)

Lo que distingue a la arquitectura de agentes de las descritas en el Capítulo 5 es un concepto que aún no habíamos tratado: el bucle controlado por el LLM.

Todo programador ha visto bucles en su código. Por bucle, nos referimos a ejecutar el mismo código varias veces hasta que se cumpla una condición de parada. La clave en la arquitectura de agentes es que el propio LLM controle esta condición: es decir, que decida cuándo dejar de iterar.

El ciclo que se ejecuta incluye generalmente:

- Planificar una o varias acciones
- Ejecutar dichas acciones

Retomando el ejemplo anterior, ahora ejecutamos la herramienta `search` con la entrada *30th president of the United States*, que produce esta salida:

> Calvin Coolidge (nacido como John Calvin Coolidge Jr.; 4 de julio de 1872 – 5 de enero de 1933) fue un abogado y político estadounidense que se desempeñó como el 30º presidente de los Estados Unidos de 1923 a 1929.

Luego volvemos a ejecutar el prompt, con una pequeña adición:

---

**Herramientas:**  
- `search`: consulta en la web.  
- `calculator`: evalúa expresiones matemáticas.  
- `output`: finaliza la interacción. Úsala cuando tengas la respuesta final.

Si necesitas herramientas para llegar a la respuesta, escribe la lista en formato CSV, con el encabezado: `tool,input`.

Piensa paso a paso; si necesitas múltiples pasos, regresa solo el primero.

**Pregunta:**  
¿Cuántos años tenía el 30º presidente de los Estados Unidos cuando murió?

**Respuesta esperada:**

```
tool,input  
search,30th president of the United States  
```

**Resultado de `search`:**  
Calvin Coolidge (nacido como John Calvin Coolidge Jr.; 4 de julio de 1872 – 5 de enero de 1933)...

**Siguiente entrada:**

```
tool,input  
calculator,1933 - 1872  
```

---

Observa que agregamos dos cosas:

1. Una herramienta `output` que el modelo debe usar cuando tenga la respuesta final. Nosotros la usaremos como señal para detener el bucle.
2. El resultado de la herramienta en la iteración previa, simplemente con el nombre de la herramienta y su salida en texto. Esto permite que el LLM avance al siguiente paso en la interacción. En otras palabras, le estamos diciendo: “Aquí tienes los resultados que pediste, ¿qué quieres hacer ahora?”

Continuamos con una tercera iteración:

```
tool,input  
search,30th president of the United States  
```

Resultado del `search`:  
> Calvin Coolidge (4 de julio de 1872 – 5 de enero de 1933)...

```
tool,input  
calculator,1933 - 1872  
```

Resultado del `calculator`:  
> 61

```
tool,input  
output,61  
```

---

Con el resultado del `calculator`, el LLM ya tiene suficiente información para proporcionar la respuesta final, así que selecciona la herramienta `output` y escribe “61” como respuesta final.

Esto es lo que hace tan útil a la arquitectura de agentes: se le da al modelo la *agencia* para decidir. Decide qué pasos tomar, cuándo avanzar y cuándo detenerse.

Esta arquitectura, conocida como **ReAct**, fue propuesta por Shunyu Yao y colaboradores. El resto del capítulo explora cómo mejorar el rendimiento de esta arquitectura, motivado por el ejemplo del asistente de correos electrónicos del Capítulo 5.

Pero primero, veamos cómo se ve la implementación básica de la arquitectura de agentes usando un modelo conversacional y LangGraph.




## Construyendo un Agente con LangGraph

Para este ejemplo, necesitamos instalar dependencias adicionales para la herramienta de búsqueda que elegimos: DuckDuckGo. Para instalarla en Python:

```bash
pip install duckduckgo-search
```

Una vez hecho esto, pasemos al código para implementar la arquitectura del agente.

Estamos utilizando dos herramientas en este ejemplo: una herramienta de búsqueda y una calculadora. Pero podrías agregar más o reemplazar las que usamos fácilmente. En el ejemplo de Python, también se muestra cómo crear una herramienta personalizada.

Usamos dos funciones convenientes que vienen con LangGraph:

- `ToolNode` actúa como un nodo en nuestro grafo; ejecuta las llamadas a herramientas solicitadas en el mensaje más reciente de la IA que se encuentra en el estado y devuelve un `ToolMessage` con los resultados de cada herramienta. También maneja excepciones que puedan surgir en las herramientas: si una herramienta falla, el error se convierte en un mensaje que se pasa al modelo LLM, el cual decidirá qué hacer con ese error.
  
- `tools_condition` funciona como una función de borde condicional. Analiza el mensaje más reciente generado por el modelo y decide si hay herramientas que ejecutar. Si es así, dirige el flujo hacia el nodo de herramientas. Si no hay herramientas que ejecutar, finaliza el grafo.

Es importante notar que este grafo tiene un **bucle** entre los nodos del modelo y de las herramientas. Es decir, el propio modelo es quien decide cuándo terminar la ejecución, lo cual es una característica clave de la arquitectura de agentes. Siempre que se programe un bucle en LangGraph, probablemente se utilizará un borde condicional como este, ya que permite definir la condición de salida del bucle.

Ahora veamos cómo se comporta con el ejemplo que vimos anteriormente:

```python
input = {
    "messages": [
        HumanMessage("¿Cuántos años tenía el 30º presidente de los Estados Unidos cuando murió?")
    ]
}
for c in graph.stream(input):
    print(c)
```

### Recorrido del resultado:

1. **Primero**, se ejecuta el nodo del modelo, que decide llamar a la herramienta `duckduckgo_search`. Como resultado, el borde condicional redirige la ejecución al nodo de herramientas.

2. **Luego**, `ToolNode` ejecuta la herramienta de búsqueda y obtiene resultados que contienen directamente la respuesta: “Edad y año de fallecimiento: 5 de enero de 1933 (60 años)”.

3. **Después**, se llama nuevamente al modelo, ahora con los resultados de la búsqueda como el mensaje más reciente, y este produce la respuesta final sin llamar a más herramientas. Por lo tanto, el borde condicional finaliza el grafo.



In [9]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

@tool
def calculator(query: str) -> str:
    """Una calculadora simple. La entrada debe ser una expresión matemática."""
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOpenAI(temperature=0.1).bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages": res}

builder = StateGraph(State)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "model")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()





### Explicación del Código y el Flujo de Trabajo

Este código implementa un flujo de trabajo que involucra un modelo de lenguaje (`ChatOpenAI`) y varias herramientas (como **DuckDuckGoSearchRun** y una calculadora personalizada). Vamos a desglosar cómo se integran las herramientas y cómo se establece un ciclo entre el modelo y las herramientas utilizando un grafo.

#### 1. **Uso de herramientas con el modelo:**

El código utiliza **`DuckDuckGoSearchRun`** como una herramienta de búsqueda, junto con una **calculadora** para realizar operaciones matemáticas. Estas herramientas son esenciales para que el modelo pueda interactuar con el entorno más allá de solo generar texto. Al asociar estas herramientas con el modelo de lenguaje, el modelo puede ejecutar funciones adicionales (como hacer una búsqueda o calcular una expresión) cuando sea necesario.

```python
search = DuckDuckGoSearchRun()  # Herramienta de búsqueda
tools = [search, calculator]   # Lista de herramientas disponibles
```

**¿Qué hace esto?**
- **`DuckDuckGoSearchRun`** es la herramienta de búsqueda que se usará para hacer consultas a través de DuckDuckGo.
- **`tools`** es una lista que contiene tanto la herramienta de búsqueda como la calculadora. En este momento, el modelo puede acceder a estas herramientas cuando el flujo del grafo lo requiera.

#### 2. **Vinculando las herramientas al modelo:**

```python
model = ChatOpenAI(temperature=0.1).bind_tools(tools)
```

**¿Qué hace esto?**
- **`bind_tools(tools)`** es un método que asocia las herramientas definidas en la lista `tools` (que incluye el `DuckDuckGoSearchRun` y la calculadora) al modelo de lenguaje `ChatOpenAI`.
- Esto le permite al modelo invocar estas herramientas en su proceso de generar respuestas, según lo determine el flujo del grafo.

**Por qué es importante:**
- Sin esta vinculación, el modelo no tendría acceso directo a las herramientas y no podría utilizarlas cuando el grafo lo requiera. `bind_tools` asegura que el modelo puede llamar a las herramientas dentro del ciclo de ejecución.

#### 3. **Condicionales en el grafo:**

```python
builder.add_conditional_edges("model", tools_condition)
```

**¿Qué hace esto?**
- **`add_conditional_edges`** agrega un borde (o "edge") condicional entre el nodo `"model"` y los nodos de herramientas.
- **`tools_condition`** es una función que determina si el modelo necesita ejecutar alguna herramienta (como la búsqueda o la calculadora). Si el modelo indica que debe usar alguna de las herramientas, el grafo llevará el flujo de ejecución al nodo de herramientas.

**¿Por qué es importante?**
- Esta función permite que el flujo del grafo sea dinámico. Solo se ejecutan las herramientas cuando el modelo las necesita, y esto se decide mediante la condición definida en `tools_condition`. Así, el grafo no ejecuta herramientas innecesarias, manteniendo el flujo eficiente.

#### 4. **Bucle entre herramientas y modelo:**

```python
builder.add_edge("tools", "model")
```

**¿Qué hace esto?**
- Este borde conecta el nodo de herramientas (`"tools"`) de vuelta al nodo del modelo (`"model"`).
- **¿Por qué?** Esto permite que el flujo regrese al nodo del modelo después de ejecutar alguna herramienta, como realizar una búsqueda o calcular algo. De esta manera, el modelo puede tomar los resultados de la herramienta y generar una nueva respuesta.

**¿Por qué es importante?**
- Esta conexión establece un ciclo entre los nodos del modelo y las herramientas. El flujo vuelve al modelo después de cada uso de las herramientas para asegurar que el modelo pueda procesar y generar una respuesta con la nueva información obtenida de las herramientas. 
- Este ciclo puede repetirse varias veces, dependiendo de si el modelo necesita más interacciones con las herramientas o si puede generar la respuesta final.

#### Resumen del Flujo

1. **Inicio del grafo**: El flujo comienza en el nodo `"model"`, donde el modelo genera una respuesta.
2. **Condición para herramientas**: Si el modelo necesita usar alguna herramienta, el flujo se mueve al nodo `"tools"`, donde se ejecutan las herramientas necesarias.
3. **Volver al modelo**: Después de ejecutar la herramienta, el flujo regresa al nodo `"model"`, y el modelo utiliza los resultados de la herramienta para generar una nueva respuesta.
4. **Repetir o finalizar**: Este ciclo se repite hasta que el modelo decide que no necesita más herramientas y se puede generar la respuesta final.

Este diseño permite que el modelo interactúe dinámicamente con el entorno, haciendo uso de herramientas cuando sea necesario y manteniendo el flujo de ejecución eficiente.



In [10]:
# Ejemplo de uso:
from langchain_core.messages import HumanMessage

input = {
    "messages": [
        HumanMessage(
            "¿Cuántos años tenía el 30° presidente de los Estados Unidos cuando murió?"
        )
    ]
}

for step in graph.stream(input):
    print(step)


{'model': {'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dZeiPIcICUwM8MWS8owQD0j2', 'function': {'arguments': '{"query":"Age of 30th president of the United States at death"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 126, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BNon3L57wrYZ5kcOmvAfIUODFykEm', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a82c82b4-2fc2-46bf-88f3-6c58d7f91aac-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'Age of 30th president of the United States at death'}, 'id': 'call_dZeiPIcICUwM8MWS8owQD0j2', 'type': 'tool_call'}], usage_metadata={'inp

In [6]:
# ! pip install langchain-community duckduckgo-search

Aquí hay algunas cosas a notar:

Estamos utilizando dos herramientas en este ejemplo: una herramienta de búsqueda y una herramienta de calculadora, pero podrías agregar fácilmente más o reemplazar las que usamos. En el ejemplo de Python, también puedes ver un ejemplo de cómo crear una herramienta personalizada.

Hemos utilizado dos funciones de conveniencia que vienen con LangGraph. ToolNode sirve como un nodo en nuestro grafo; ejecuta las llamadas a las herramientas solicitadas en el último mensaje de IA encontrado en el estado y devuelve un ToolMessage con los resultados de cada una. ToolNode también maneja las excepciones que pueden ser levantadas por las herramientas, utilizando el mensaje de error para construir un ToolMessage que luego se pasa al LLM, el cual puede decidir qué hacer con el error.

tools_condition sirve como una función de borde condicional que observa el último mensaje de IA en el estado y redirige al nodo de herramientas si hay alguna herramienta que ejecutar. De lo contrario, termina el grafo.

Finalmente, observa que este grafo hace un ciclo entre los nodos del modelo y las herramientas. Es decir, el modelo mismo se encarga de decidir cuándo terminar el cálculo, lo cual es un atributo clave de la arquitectura del agente. Siempre que codifiquemos un ciclo en LangGraph, probablemente querremos usar un borde condicional, ya que esto te permite definir la condición de parada para cuando el grafo debe salir del ciclo y dejar de ejecutarse.

In [14]:
import ast
import json
from typing import Annotated, TypedDict
from langchain_core.messages import HumanMessage, BaseMessage

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Herramienta de calculadora
@tool
def calculator(query: str) -> str:
    """A simple calculator tool. Input should be a mathematical expression."""
    return str(ast.literal_eval(query))

# Herramientas disponibles
search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOpenAI(temperature=0.1).bind_tools(tools)

# Estado del grafo
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Nodo del modelo
def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages": res}

# Grafo
builder = StateGraph(State)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "model")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()

input = {
    "messages": [
        HumanMessage(
            "¿Cuántos años tenía el 30° presidente de los Estados Unidos cuando murió?"
        )
    ]
}

# Función para serializar mensajes individuales
def serialize_message(msg):
    if isinstance(msg, BaseMessage):
        return msg.dict()
    elif isinstance(msg, tuple) and isinstance(msg[1], BaseMessage):
        return { "name": msg[0], **msg[1].dict() }
    else:
        return str(msg)


# Ejecutar el agente y mostrar los pasos
print("\nThe output:\n")
for step in graph.stream(input_data):
    for key, value in step.items():
        print(f"\nStep: {key}")
        if "messages" in value:
            msgs = value["messages"]
            if isinstance(msgs, list):
                serialized = [serialize_message(m) for m in msgs]
            else:
                serialized = serialize_message(msgs)
            print(json.dumps({key: {"messages": serialized}}, indent=4))



The output:


Step: model
{
    "model": {
        "messages": {
            "content": "",
            "additional_kwargs": {
                "tool_calls": [
                    {
                        "id": "call_gzq6TjuwPUA9W0eoLOZNzWPr",
                        "function": {
                            "arguments": "{\"query\":\"30th president of the United States age at death\"}",
                            "name": "duckduckgo_search"
                        },
                        "type": "function"
                    }
                ],
                "refusal": null
            },
            "response_metadata": {
                "token_usage": {
                    "completion_tokens": 27,
                    "prompt_tokens": 118,
                    "total_tokens": 145,
                    "completion_tokens_details": {
                        "accepted_prediction_tokens": 0,
                        "audio_tokens": 0,
                        "reasoning_tokens": 0,


/tmp/ipykernel_675/669740557.py:55: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return msg.dict()



Step: tools
{
    "tools": {
        "messages": [
            {
                "content": "Calvin Coolidge (born John Calvin Coolidge Jr.; [1] / \u02c8 k u\u02d0 l \u026a d\u0292 / KOOL-ij; July 4, 1872 - January 5, 1933) was the 30th president of the United States, serving from 1923 to 1929.A Republican lawyer from Massachusetts, he previously served as the 29th vice president from 1921 to 1923 under President Warren G. Harding, and as the 48th governor of Massachusetts from 1919 to 1921. Calvin Coolidge was the 30th president of the United States (1923-29). Coolidge acceded to the presidency after the death in office of Warren G. Harding, just as the Harding scandals were coming to light. He restored integrity to the executive branch while continuing Harding's conservative pro-business policies. The White House, official residence of the president of the United States. The president of the United States is the head of state and head of government of the United States, [1] indirect

/tmp/ipykernel_675/669740557.py:55: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return msg.dict()



Step: model
{
    "model": {
        "messages": {
            "content": "Calvin Coolidge, the 30th president of the United States, died on January 5, 1933. He was born on July 4, 1872. To calculate his age at the time of his death, we need to subtract his birth year from the year of his death:\n\n1933 (year of death) - 1872 (birth year) = 61 years old\n\nCalvin Coolidge was 61 years old when he died.",
            "additional_kwargs": {
                "refusal": null
            },
            "response_metadata": {
                "token_usage": {
                    "completion_tokens": 98,
                    "prompt_tokens": 584,
                    "total_tokens": 682,
                    "completion_tokens_details": {
                        "accepted_prediction_tokens": 0,
                        "audio_tokens": 0,
                        "reasoning_tokens": 0,
                        "rejected_prediction_tokens": 0
                    },
                    "prompt_tokens_d

/tmp/ipykernel_675/669740557.py:55: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return msg.dict()




**Llamando siempre a una herramienta primero**  
En la arquitectura estándar de agentes, el LLM siempre es llamado para decidir qué herramienta usar a continuación. Este enfoque tiene una ventaja clara: le da al LLM una flexibilidad total para adaptar el comportamiento de la aplicación a cada consulta del usuario. Pero esta flexibilidad tiene un costo: la imprevisibilidad. Por ejemplo, si tú, como desarrollador de la aplicación, sabes que la herramienta de búsqueda siempre debe ser llamada primero, eso en realidad puede beneficiar tu aplicación:

- Reducirá la latencia general, ya que se omitirá la primera llamada al LLM que solo generaría esa solicitud para usar la herramienta de búsqueda.
- Evitará que el LLM decida erróneamente que no necesita llamar a la herramienta de búsqueda para algunas consultas.

Por otro lado, si tu aplicación no tiene una regla clara como “siempre debes llamar a esta herramienta primero”, imponer esa restricción en realidad podría empeorar la aplicación.

Veamos cómo se hace esto:

In [15]:

### Código en Python
import ast
from typing import Annotated, TypedDict
from uuid import uuid4

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import AIMessage, ToolCall
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

@tool
def calculator(query: str) -> str:
    """Una herramienta de calculadora simple. El input debe ser una expresión matemática."""
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOpenAI(temperature=0.1).bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages": res}

def first_model(state: State) -> State:
    query = state["messages"][-1].content
    search_tool_call = ToolCall(
        name="duckduckgo_search", args={"query": query}, id=uuid4().hex
    )
    return {"messages": AIMessage(content="", tool_calls=[search_tool_call])}

builder = StateGraph(State)
builder.add_node("first_model", first_model)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "first_model")
builder.add_edge("first_model", "tools")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()




**Observa las diferencias con respecto a la sección anterior:**

Ahora, comenzamos todas las invocaciones llamando a `first_model`, que no llama a ningún modelo de lenguaje. Solo crea una llamada a la herramienta de búsqueda, usando el mensaje del usuario tal cual como consulta. La arquitectura anterior habría hecho que el LLM generara esta llamada a herramienta (u otra respuesta que considerara mejor).

Después de eso, pasamos a `tools`, que es idéntico al ejemplo anterior, y de ahí procedemos al nodo del agente como antes.

Ahora veamos un ejemplo de salida, usando la misma consulta que antes:

---

### Código en Python

In [16]:


input = {
    "messages": [
        HumanMessage("""How old was the 30th president of the United States 
            when he died?""")
    ]
}
for c in graph.stream(input):
    print(c)




{'first_model': {'messages': AIMessage(content='', additional_kwargs={}, response_metadata={}, id='ebc360bd-7d62-4120-8e1d-125b07e41f9b', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'How old was the 30th president of the United States \n            when he died?'}, 'id': '5f64ab25e4274ef4a7cbca819f375572', 'type': 'tool_call'}])}}
{'tools': {'messages': [ToolMessage(content='Calvin Coolidge (born John Calvin Coolidge Jr.; [1] / ˈ k uː l ɪ dʒ / KOOL-ij; July 4, 1872 - January 5, 1933) was the 30th president of the United States, serving from 1923 to 1929.A Republican lawyer from Massachusetts, he previously served as the 29th vice president from 1921 to 1923 under President Warren G. Harding, and as the 48th governor of Massachusetts from 1919 to 1921. The White House, official residence of the president of the United States. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via

---

**Esta vez, nos saltamos la llamada inicial al LLM.**  
Primero fuimos al nodo `first_model`, que directamente devolvió una llamada a la herramienta de búsqueda.  
De ahí seguimos el flujo anterior —es decir, ejecutamos la herramienta de búsqueda y finalmente regresamos al nodo del modelo para generar la respuesta final.

---

A continuación, veamos qué puedes hacer cuando tienes **muchas herramientas** que deseas poner a disposición del LLM.



### Tratando con Muchas Herramientas

Los modelos de lenguaje (LLMs) no son perfectos y tienden a tener dificultades cuando se les presentan muchas opciones o demasiada información en un mismo prompt.  
Estas limitaciones también afectan su capacidad para planear cuál debería ser la siguiente acción a tomar.  
Cuando se les da muchas herramientas (por ejemplo, más de 10), su rendimiento al elegir la herramienta correcta empieza a disminuir.

#### ¿La solución?

Reducir el número de herramientas entre las que el LLM puede elegir.  
Pero, ¿qué pasa si realmente tienes muchas herramientas que quieres usar dependiendo del tipo de consulta?

Una solución elegante es usar un paso de **RAG (Recuperación Augmentada con Generación)** para **preseleccionar** las herramientas más relevantes para la consulta actual y luego alimentar al LLM solo con ese subconjunto, en lugar de todo el arsenal de herramientas.  

Esto también ayuda a reducir el costo de invocación del LLM (los modelos comerciales cobran con base en la longitud del prompt y la respuesta).

**Nota:** este paso RAG añade latencia, así que solo deberías implementarlo cuando observes que el rendimiento cae después de agregar muchas herramientas.

---

### Código en Python


AttributeError: 'function' object has no attribute 'description'

In [17]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_core.vectorstores.in_memory import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

@tool
def calculator(query: str) -> str:
    """A simple calculator tool. Input should be a mathematical expression."""
    return ast.literal_eval(query)

search = DuckDuckGoSearchRun()
tools = [search, calculator]

embeddings = OpenAIEmbeddings()
model = ChatOpenAI(temperature=0.1)

tools_retriever = InMemoryVectorStore.from_documents(
    [Document(tool.description, metadata={"name": tool.name}) for tool in tools],
    embeddings,
).as_retriever()

class State(TypedDict):
    messages: Annotated[list, add_messages]
    selected_tools: list[str]

def model_node(state: State) -> State:
    selected_tools = [
        tool for tool in tools if tool.name in state["selected_tools"]
    ]
    res = model.bind_tools(selected_tools).invoke(state["messages"])
    return {"messages": res}

def select_tools(state: State) -> State:
    query = state["messages"][-1].content
    tool_docs = tools_retriever.invoke(query)
    return {"selected_tools": [doc.metadata["name"] for doc in tool_docs]}

builder = StateGraph(State)
builder.add_node("select_tools", select_tools)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "select_tools")
builder.add_edge("select_tools", "model")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()


**NOTA:**  
Esto es muy similar a la arquitectura de agentes regular.  
La única diferencia es que **hacemos una parada en el nodo `select_tools`** antes de entrar al ciclo del agente como tal.  
Después de eso, funciona igual que la arquitectura de agentes que vimos antes.

Veamos un ejemplo de salida para la misma consulta anterior:


In [21]:
from langchain_core.tools import tool

# Definir herramientas
@tool
def calculator(query: str) -> str:
    """Una calculadora simple. La entrada debe ser una expresión matemática."""
    return str(eval(query))

@tool
def greet(name: str) -> str:
    """Devuelve un saludo amistoso."""
    return f"Hola, {name}!"

# Lista de herramientas
tools = [calculator, greet]

# Mostrar nombre y descripción de cada herramienta
for tool in tools:
    print(f"Nombre: {tool.name}")
    print(f"Descripción: {tool.description}")
    print("-" * 40)


Nombre: calculator
Descripción: Una calculadora simple. La entrada debe ser una expresión matemática.
----------------------------------------
Nombre: greet
Descripción: Devuelve un saludo amistoso.
----------------------------------------


In [18]:
input = {
    "messages": [
        HumanMessage("""How old was the 30th president of the United States when 
            he died?""")
    ]
}
for c in graph.stream(input):
    print(c)


{'select_tools': {'selected_tools': ['duckduckgo_search', 'calculator']}}
{'model': {'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rCd89Je3KZPTHyTwHfIlZKRX', 'function': {'arguments': '{"query":"30th president of the United States age at death"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 120, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BNpoFhrWlrY2CsnTqwEr1KoxehkRJ', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e551b605-e423-417f-92af-87fb5601fddb-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '30th president of the United States age at death'}, 'id': 'call_rC



Observa cómo lo primero que hicimos fue **consultar al recuperador** (`retriever`) para obtener las herramientas **más relevantes** para la consulta actual del usuario.  
Luego, procedimos con la **arquitectura de agente regular**.

---

### Resumen

Este capítulo introdujo el concepto de **agencia** y discutió lo necesario para hacer que una aplicación basada en un LLM sea **agentiva**:  
darle al LLM la **capacidad de decidir entre múltiples opciones**, usando información externa.

Recorrimos la arquitectura estándar de un agente construida con **LangGraph**, y exploramos dos extensiones útiles:

- **Cómo llamar siempre primero a una herramienta específica**.
- **Cómo manejar múltiples herramientas** de forma eficiente.

El **Capítulo 7** explorará más extensiones para esta arquitectura de agentes.

